In [1]:
import pandas as pd
import numpy as np
import time

In [16]:
team_data = pd.read_csv('inputData/team_data1.csv')
team_data.head()

,year,team_name,wins,loss,win_rates,coach_off_times,coach_wins_rate,coach_champions
0,1985,波士顿凯尔特人,67,15,81.7%,9,67.4,2
1,1985,密尔沃基雄鹿,57,25,69.5%,18,55.7,0
2,1985,费城76人,54,28,65.9%,2,43.0,0
3,1985,亚特兰大老鹰,50,32,61.0%,11,54.9,0
4,1985,底特律活塞,46,36,56.1%,12,59.3,2


In [17]:
team_data.columns = ['season','rival','wins','loss','rival_rates','rival_coach_offs','rival_coach_rates','coach_campions']

In [18]:
team_data.head()

,season,rival,wins,loss,rival_rates,rival_coach_offs,rival_coach_rates,coach_campions
0,1985,波士顿凯尔特人,67,15,81.7%,9,67.4,2
1,1985,密尔沃基雄鹿,57,25,69.5%,18,55.7,0
2,1985,费城76人,54,28,65.9%,2,43.0,0
3,1985,亚特兰大老鹰,50,32,61.0%,11,54.9,0
4,1985,底特律活塞,46,36,56.1%,12,59.3,2


In [19]:
teams = ['马刺', '小牛', '太阳', '勇士', '骑士', '爵士', '掘金', '开拓者', '快船', '篮网', '超音速',
       '公牛', '火箭', '活塞', '尼克斯', '雄鹿', '子弹', '国王', '步行者', '凯尔特', '76人',
       '老鹰', '热火', '黄蜂', '魔术', '森林狼', '灰熊', '猛龙', '奇才', '山猫', '雷霆', '鹈鹕',
       '独行侠','湖人']

In [20]:
#次函数得到team_data里队伍的对名，去掉前面的地名
def get_team(x):
    for i in teams:
        if(i in x):
            return i

In [21]:
team_data['rival'] = team_data['rival'].apply(get_team)
team_data['wins-loss'] = team_data['wins'] - team_data['loss']

In [22]:
#去除热火人队，因为我们需要预测湖人的比赛结果，对手不可能是湖人队
team_data['rival_rates'] = team_data['rival_rates'].str.strip('%').astype(float) /100#将百分号去掉转化为float类型
heat_info = team_data.loc[team_data['rival']=='热火']
team_data = team_data.loc[team_data['rival']!='热火']
team_data.head()

,season,rival,wins,loss,rival_rates,rival_coach_offs,rival_coach_rates,coach_campions,wins-loss
0,1985,凯尔特,67,15,0.817,9,67.4,2,52
1,1985,雄鹿,57,25,0.695,18,55.7,0,32
2,1985,76人,54,28,0.659,2,43.0,0,26
3,1985,老鹰,50,32,0.610,11,54.9,0,18
4,1985,活塞,46,36,0.561,12,59.3,2,10


In [23]:
heat_data = pd.read_csv('inputData/heat_data.csv')

In [24]:
heat_data = heat_data.iloc[:,0:5]
heat_data.head()

,team_name,game_time,game_rst,host_guest,rival
0,迈阿密热火,1988-11-05,负,主,快船111-91热火
1,迈阿密热火,1988-11-08,负,客,小牛92-88热火
2,迈阿密热火,1988-11-09,负,客,马刺117-93热火
3,迈阿密热火,1988-11-11,负,主,火箭121-100热火
4,迈阿密热火,1988-11-15,负,主,凯尔特84-65热火


In [25]:
heat_data['rival'] = heat_data['rival'].str.extract('(\d{0,2}.*?)\d')#提取出对手队名

In [26]:
#此函数求出湖人队每场的比赛所在的赛季，也是用于连接
def get_season(game_time):
    ymd_lst = game_time.split('-')
    year = ymd_lst[0]
    month = int(ymd_lst[1])
    day = ymd_lst[2]
    if(month>9):
        return int(year)
    else:
        return int(year) -1

In [27]:
heat_data['season'] = heat_data['game_time'].apply(get_season)

In [28]:
tmp_data = list([i for i in np.array(heat_data.game_time)[:-1]])
tmp_data.insert(0,'1988-11-3')
heat_data['game_time1'] = tmp_data
heat_data['game_time'] = pd.to_datetime(heat_data['game_time'])
heat_data['game_time1'] = pd.to_datetime(heat_data['game_time1'])
heat_data['days'] = heat_data['game_time'] - heat_data['game_time1']
heat_data.to_csv('inputData/days1.csv',index=False)#这段代码是为了求出比赛相隔的天数，生产一个新的文件，在去处理文件，重新读取即可

In [29]:
heat_data = pd.read_csv('inputData/days1.csv')

In [30]:
heat_data['game_month'] = heat_data['game_time'].str.extract('-(.*?)-')

In [31]:
heat_data.drop(['game_time','game_time1'],axis=1,inplace=True)

In [32]:
heat_data = heat_data.loc[heat_data['rival']!=' ']#去除无用数据

In [33]:
heat_info.head()

,season,rival,wins,loss,rival_rates,rival_coach_offs,rival_coach_rates,coach_campions,wins-loss
81,1988,热火,15,67,0.183,0,29.6,0,-52
106,1989,热火,18,64,0.220,0,29.6,0,-46
134,1990,热火,24,58,0.293,0,29.6,0,-34
155,1991,热火,38,44,0.463,8,41.7,0,-6
185,1992,热火,36,46,0.439,8,41.7,0,-10


In [34]:
heat_info.drop(['rival','coach_campions','wins','loss'],axis=1,inplace=True)

In [35]:
heat_info.columns = ['season','rates','coach_offs','coach_rates','win-loss']

In [36]:
tmp_heat_data = pd.merge(heat_info,heat_data,on=['season'])
tmp_heat_data.head()

,season,rates,coach_offs,coach_rates,win-loss,team_name,game_rst,host_guest,rival,days,game_month
0,1988,0.183,0,29.6,-52,迈阿密热火,负,主,快船,2 days 00:00:00.000000000,11
1,1988,0.183,0,29.6,-52,迈阿密热火,负,客,小牛,3 days 00:00:00.000000000,11
2,1988,0.183,0,29.6,-52,迈阿密热火,负,客,马刺,1 days 00:00:00.000000000,11
3,1988,0.183,0,29.6,-52,迈阿密热火,负,主,火箭,2 days 00:00:00.000000000,11
4,1988,0.183,0,29.6,-52,迈阿密热火,负,主,凯尔特,4 days 00:00:00.000000000,11


In [37]:
team_data.rename(columns={'win-loss':'rival_win_loss'},inplace=True)
heat_machine = pd.merge(tmp_heat_data,team_data,on=['season','rival'])

In [38]:
heat_machine.head()

,season,rates,coach_offs,coach_rates,win-loss,team_name,game_rst,host_guest,rival,days,game_month,wins,loss,rival_rates,rival_coach_offs,rival_coach_rates,coach_campions,wins-loss
0,1988,0.183,0,29.6,-52,迈阿密热火,负,主,快船,2 days 00:00:00.000000000,11,21,61,0.256,10,47.7,0,-40
1,1988,0.183,0,29.6,-52,迈阿密热火,胜,客,快船,2 days 00:00:00.000000000,12,21,61,0.256,10,47.7,0,-40
2,1988,0.183,0,29.6,-52,迈阿密热火,胜,主,快船,2 days 00:00:00.000000000,02,21,61,0.256,10,47.7,0,-40
3,1988,0.183,0,29.6,-52,迈阿密热火,胜,客,快船,2 days 00:00:00.000000000,03,21,61,0.256,10,47.7,0,-40
4,1988,0.183,0,29.6,-52,迈阿密热火,负,客,马刺,1 days 00:00:00.000000000,11,21,61,0.256,18,54.8,1,-40


In [39]:
heat_machine.drop(['season','team_name'],inplace=True,axis=1)

In [40]:
heat_machine = heat_machine.loc[heat_machine['rival']!= '超音速']

In [41]:
heat_machine = heat_machine.loc[heat_machine['rival']!= '子弹']

In [42]:
heat_machine['rival'].unique()

array(['快船', '马刺', '火箭', '凯尔特', '勇士', '湖人', '雄鹿', '骑士', '黄蜂', '开拓者', '国王',
       '掘金', '公牛', '爵士', '活塞', '太阳', '步行者', '老鹰', '篮网', '76人', '尼克斯',
       '魔术', '森林狼', '灰熊', '猛龙', '奇才', '雷霆', '鹈鹕', '独行侠'], dtype=object)

In [43]:
teams = heat_machine['rival'].unique()

In [44]:
tmp_dict = {}
value = 1
for i in teams:
    tmp_dict[i] = value
    value +=1

In [45]:
heat_machine['rival_id'] = heat_machine['rival'].map(lambda x:tmp_dict[x])

In [49]:
tmp_dict

{'快船': 1,
 '马刺': 2,
 '火箭': 3,
 '凯尔特': 4,
 '勇士': 5,
 '湖人': 6,
 '雄鹿': 7,
 '骑士': 8,
 '黄蜂': 9,
 '开拓者': 10,
 '国王': 11,
 '掘金': 12,
 '公牛': 13,
 '爵士': 14,
 '活塞': 15,
 '太阳': 16,
 '步行者': 17,
 '老鹰': 18,
 '篮网': 19,
 '76人': 20,
 '尼克斯': 21,
 '魔术': 22,
 '森林狼': 23,
 '灰熊': 24,
 '猛龙': 25,
 '奇才': 26,
 '雷霆': 27,
 '鹈鹕': 28,
 '独行侠': 29}

In [46]:
heat_machine.to_csv('inputData/heat_machine.csv',index=False)

In [13]:
heat_data = pd.read_csv('inputData/heat_machine.csv')

In [4]:
heat_data['racer'] = heat_data['rival'] 

In [5]:
heat_data.head()

,rates,coach_offs,coach_rates,win-loss,game_rst,host_guest,rival,days,game_month,wins,loss,rival_rates,rival_coach_offs,rival_coach_rates,coach_campions,wins-loss,rival_id,racer
0,0.183,0,29.6,-52,负,主,快船,2 days 00:00:00.000000000,11,21,61,0.256,10,47.7,0,-40,1,快船
1,0.183,0,29.6,-52,胜,客,快船,2 days 00:00:00.000000000,12,21,61,0.256,10,47.7,0,-40,1,快船
2,0.183,0,29.6,-52,胜,主,快船,2 days 00:00:00.000000000,2,21,61,0.256,10,47.7,0,-40,1,快船
3,0.183,0,29.6,-52,胜,客,快船,2 days 00:00:00.000000000,3,21,61,0.256,10,47.7,0,-40,1,快船
4,0.183,0,29.6,-52,负,客,马刺,1 days 00:00:00.000000000,11,21,61,0.256,18,54.8,1,-40,2,马刺


In [16]:
heat_data.drop(['wins','loss'],axis=1,inplace=True)

In [17]:
heat_data.to_csv('inputData/heat_machine.csv',index=False)